In [ ]:
import pandas as pd
import re
import numpy as np

Data preprocessing, converting to numpy array and dividing into train, test, validation dataset. Data location of both glove matrix and sentiment(2) has to be changed.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv (r'/content/drive/MyDrive/sentiment (2).csv') #change the value to your file location while running
print (df)

In [ ]:
reviews = df['review']
labels = np.array(df['senti_score'])

In [ ]:
for label in range(len(labels)):
  if float(labels[label])<=0.2:
    labels[label] = 0
  elif float(labels[label])<=0.4:
    labels[label] = 1
  elif float(labels[label])<=0.6:
    labels[label] = 2
  elif float(labels[label])<=0.8:
    labels[label] = 3
  elif float(labels[label])<=1.0:
    labels[label] = 4

In [ ]:
reviews = reviews.str.lower()
for i in range(df.shape[0]):
  reviews[i] = re.sub(r'[^\w\s]', '', reviews[i])

In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('/content/drive/MyDrive/glove.6B.100d.txt', encoding="utf8")   #change the value to your file location while running

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [ ]:
maxlen = 32 #length of review set here

In [ ]:
embedding_matrix = np.empty((len(reviews),maxlen,100))
j=0
k=0
k2=0
for sent in reviews:
  sent_embed = np.empty((maxlen,100))
  i=0
  for word in sent.split():
    if i==maxlen:
      break
    
    try:
      sent_embed[i] = np.array(embeddings_dictionary[word]).reshape((1,100))
    except KeyError: 
      sent_embed[i] = np.zeros((1,100))
      k+=1
    print(sent_embed.shape)
    i+=1
    k2+=1

  if i<maxlen:
    padding_matrix = np.zeros((maxlen-i,100))
    sent_embed[i:] = padding_matrix
  embedding_matrix[j] = sent_embed
  j+=1

In [ ]:
X = embedding_matrix

In [ ]:
y = np.empty((len(reviews),maxlen))
com = np.ones((1,maxlen))
i=0
for label in labels:
  y[i] = np.array(label*com)
  i+=1
y = y.astype(int)

In [ ]:
# fp = open('/content/Train-Dev_test split.txt', 'r')
sent=np.loadtxt('/content/drive/MyDrive/Train-Dev_test split.txt',delimiter=',',dtype=str)

In [140]:
X_train = np.empty((len(reviews),maxlen,100))
X_test = np.empty((len(reviews),maxlen,100)) 
X_val = np.empty((len(reviews),maxlen,100))
y_train = np.empty((len(reviews),maxlen))
y_test = np.empty((len(reviews),maxlen))
y_val = np.empty((len(reviews),maxlen))
i=0
j=0
b=0

In [141]:
for k in range(len(reviews)):
  if sent[k][1]=='1':
    X_train[i] = X[max(11285,int(sent[k][0])-1)]
    y_train[i] = y[max(11285,int(sent[k][0])-1)]
    i+=1
  elif sent[k][1]=='2':
    X_val[b] = X[max(11285,int(sent[k][0])-1)]
    y_val[b] = y[max(11285,int(sent[k][0])-1)]
    b+=1
  elif sent[k][1]=='3':
    X_test[j] = X[max(11285,int(sent[k][0])-1)]
    y_test[j] = y[max(11285,int(sent[k][0])-1)]
    j+=1
y_train = y_train.astype(int)
y_val = y_val.astype(int)
y_test = y_test.astype(int)

Model

In [ ]:
import numpy as np
import sys
from datetime import datetime

Multiplication gate for the network, multiply weights with input/H(t-1)

In [ ]:
class MultiplyGate:
    def forward(self,W, x):
        return np.dot(W, x)
    def backward(self, W, x, dz):
        dW = np.asarray(np.dot(np.transpose(np.asmatrix(dz)), np.asmatrix(x)))
        dx = np.dot(np.transpose(W), dz)
        return dW, dx

Addition gate for the network, add terms in the forward prop of RNN

In [ ]:
class AddGate:
    def forward(self, x1, x2):
        return x1 + x2
    def backward(self, x1, x2, dz):
        dx1 = dz * np.ones_like(x1)
        dx2 = dz * np.ones_like(x2)
        return dx1, dx2

In [ ]:
class Sigmoid:
    def forward(self, x):
        return 1.0 / (1.0 + np.exp(-x))
    def backward(self, x, top_diff):
        output = self.forward(x)
        return (1.0 - output) * output * top_diff

In [ ]:
class Tanh:
    def forward(self, x):
        return np.tanh(x)
    def backward(self, x, top_diff):
        output = self.forward(x)
        return (1.0 - np.square(output)) * top_diff

In [ ]:
class Softmax:
  def predict(self, x):
    log_c = np.max(x, axis=x.ndim - 1, keepdims=True)
    y = np.sum(np.exp(x - log_c), axis=x.ndim - 1, keepdims=True)
    return np.exp(x - log_c)/y
  def loss(self, x, y):
    probs = self.predict(x)
    return -np.log(probs[y])
  def diff(self, x, y):
    probs = self.predict(x)
    probs[y] -= 1.0
    return probs

1 RNN layer is compiled here, both forward and backward propagation

In [ ]:
mulGate = MultiplyGate()
addGate = AddGate()
activation = Sigmoid()

class RNNLayer:
    def forward(self, x, prev_s, U, W, V):
        self.mulu = mulGate.forward(U, x)
        self.mulw = mulGate.forward(W, prev_s)
        self.add = addGate.forward(self.mulw, self.mulu)
        self.s = activation.forward(self.add)
        self.mulv = mulGate.forward(V, self.s)
        
    def backward(self, x, prev_s, U, W, V, diff_s, dmulv):
        self.forward(x, prev_s, U, W, V)
        dV, dsv = mulGate.backward(V, self.s, dmulv)
        ds = dsv + diff_s
        dadd = activation.backward(self.add, ds)
        dmulw, dmulu = addGate.backward(self.mulw, self.mulu, dadd)
        dW, dprev_s = mulGate.backward(W, prev_s, dmulw)
        dU, dx = mulGate.backward(U, x, dmulu)
        return (dprev_s, dU, dW, dV)

Entire model is compiled here, both forward and backward propagation, contains both gradient clipping and regularization

In [ ]:
class Model:
    def __init__(self, word_dim, hidden_dim=128, bptt_truncate=4):
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        self.U = np.random.uniform(-np.sqrt(1. / word_dim), np.sqrt(1. / word_dim), (hidden_dim, word_dim))
        self.W = np.random.uniform(-np.sqrt(1. / hidden_dim), np.sqrt(1. / hidden_dim), (hidden_dim, hidden_dim))
        self.V = np.random.uniform(-np.sqrt(1. / hidden_dim), np.sqrt(1. / hidden_dim), (word_dim, hidden_dim))

    def forward_propagation(self, x):
        # The total number of time steps
        T = len(x)
        layers = []
        prev_s = np.zeros(self.hidden_dim)
        # For each time step...
        for t in range(T):
            layer = RNNLayer()
            input = x[t]
            layer.forward(input, prev_s, self.U, self.W, self.V)
            prev_s = layer.s
            layers.append(layer)
        return layers

    def predict(self, x):
        output = Softmax()
        layers = self.forward_propagation(x)
        return [np.argmax(output.predict(layer.mulv)) for layer in layers]

    def calculate_loss(self, x, y):
        assert len(x) == len(y)
        output = Softmax()
        layers = self.forward_propagation(x)
        loss = 0.0
        for i, layer in enumerate(layers):
            loss += output.loss(layer.mulv, y[i])
        return loss / float(len(y))

    def calculate_total_loss(self, X, Y):
        loss = 0.0
        for i in range(len(Y)):
            loss += self.calculate_loss(X[i], Y[i])
        return loss / float(len(Y))

    def bptt(self, x, y):    #back propagation through time
        assert len(x) == len(y)
        output = Softmax()
        layers = self.forward_propagation(x)
        dU = np.zeros(self.U.shape)
        dV = np.zeros(self.V.shape)
        dW = np.zeros(self.W.shape)

        T = len(layers)
        prev_s_t = np.zeros(self.hidden_dim)
        diff_s = np.zeros(self.hidden_dim)
        for t in range(0, T):
            dmulv = output.diff(layers[t].mulv, y[t])
            input = x[t]
            dprev_s, dU_t, dW_t, dV_t = layers[t].backward(input, prev_s_t, self.U, self.W, self.V, diff_s, dmulv)
            prev_s_t = layers[t].s
            dmulv = np.zeros(self.word_dim)
            #gradient clipping is done here, 
            for i in range(t-1, max(-1, t-self.bptt_truncate-1), -1):
                input = np.zeros(self.word_dim)
                input = x[i]
                prev_s_i = np.zeros(self.hidden_dim) if i == 0 else layers[i-1].s
                dprev_s, dU_i, dW_i, dV_i = layers[i].backward(input, prev_s_i, self.U, self.W, self.V, dprev_s, dmulv)
                dU_t += dU_i
                dW_t += dW_i
            dV += dV_t
            dU += dU_t
            dW += dW_t
        return (dU, dW, dV)

    def sgd_step(self, x, y, reg_param, learning_rate):   # Regularization is implemented here, reg_param is the regularization parameter
        dU, dW, dV = self.bptt(x, y)
        self.U -= learning_rate * dU + (reg_param/maxlen)*self.U
        self.V -= learning_rate * dV + (reg_param/maxlen)*self.V
        self.W -= learning_rate * dW + (reg_param/maxlen)*self.W

    def train(self, X, Y, learning_rate, reg_param, nepoch=100, evaluate_loss_after=5):
        num_examples_seen = 0
        #initialize the vector
        m_t = np.zeros(3) 
        v_t = np.zeros(3)
        t = np.zeros(3)
        losses = []
        for epoch in range(nepoch):
            if (epoch % evaluate_loss_after == 0):
                loss = self.calculate_total_loss(X, Y)
                losses.append((num_examples_seen, loss))
                time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print("%s: Loss after num_examples_seen=%d epoch=%d: %f" % (time, num_examples_seen, epoch, loss))
                # Adjust the learning rate if loss increases
                if len(losses) > 1 and losses[-1][1] > losses[-2][1]:
                    learning_rate = learning_rate * 0.5
                    print("Setting learning rate to %f" % learning_rate)
                sys.stdout.flush()
            # For each training example...
            for i in range(len(Y)):
                self.sgd_step(X[i], Y[i], reg_param, learning_rate)
                num_examples_seen += 1
        return losses

Code for training, trained in batches of 100

In [ ]:
loss = []
word_dim = 100 # length of embedding of glove vector
hidden_dim = 128 # size of hidden dimensions
num_classes = 5  # number of classes

np.random.seed(10)
rnn = Model(word_dim, num_classes, hidden_dim)
for i in range(112):
  losses = rnn.train(X_train[i*100:100*(i+1)], y_train[i*100:100*(i+1)], reg_param = 0.02, learning_rate=0.0001, nepoch=2, evaluate_loss_after=1)
  loss.append(losses[-1][1])
  print(losses)

In [ ]:
#hyperparameter testing
# hidden size of RNN, alpha, epoch
#word embeddings are not considered as hyperparameter as pretrained glove matrix are hard to handle on colab(only size 100 used)

In [ ]:
hidden_vals = [64,128,256]
alpha_vals = [0.1,0.01,0.001]
reg_param_vals = [0, 0.02, 0.04, 0.08, 0.16, 0.32]
hidden_param = []
reg_param = []
learning_rate = []
word_dim = 100
num_classes = 5

for hidden_dim in hidden_vals:   # hidden dimensions
  np.random.seed(10)
  rnn = Model(word_dim, num_classes, hidden_dim)
  losses = rnn.train(X_val, y_val, reg_param = 0.02, learning_rate=0.005, nepoch=10, evaluate_loss_after=1)
  hidden_param.append(losses[-1][1])
  print(losses)

for lr in alpha_vals:              # learning rate
  np.random.seed(10)
  rnn = Model(word_dim, num_classes, hidden_dim)
  losses = rnn.train(X_val, y_val, reg_param = 0.02, learning_rate=lr, nepoch=10, evaluate_loss_after=1)
  learning_rate.append(losses[-1][1])
  print(losses)

for lamb in reg_param_vals:        # regularization parameter, both with regularization and without one
  np.random.seed(10)
  rnn = Model(word_dim, num_classes, hidden_dim)
  losses = rnn.train(X_val, y_val, reg_param = lamb, learning_rate=0.005, nepoch=10, evaluate_loss_after=1)
  reg_param.append(losses[-1][1])
  print(losses)

Printing values of various hyperparameters 

In [ ]:
import matplotlib.pyplot as plt
plt.title('hidden parameter')
plt.ylabel('loss')
plt.xlabel('number of nodes')
plt.plot([64,128,256], hidden_param)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.title('regularization parameter, zero corresponds to no regularization')
plt.ylabel('loss')
plt.xlabel('value of lambda')
plt.plot([0, 0.02, 0.04, 0.08, 0.16, 0.32], reg_param)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.title('learning rate')
plt.ylabel('loss')
plt.xlabel('value of alpha')
plt.plot([0.1,0.01,0.001], learning_rate)
plt.show()

Code testing , do not run

In [ ]:
said = [-0.13569,0.14029,0.0041988,-0.32062,0.012745,0.92511,-0.44523,-0.16454,0.6016,0.4267,0.26053,0.71426,0.57701,-0.09754,0.64286,-0.0002438,-0.3013,0.097057,-0.21678,-0.27131,0.30927,3.0062,-0.3179,0.28998,-0.39905,0.11234,0.19019,-0.49873,0.11857,-0.22241,-0.52668,-0.040781,-0.16783,-0.35887,0.60394,-0.069027,-0.60611,-0.22444,-0.51665,0.77766,-0.41533,-0.23924,-0.64725,0.16413,-0.41185,-0.15507,0.52085,-0.29298,0.034067,-0.71414,0.17124,0.18186,-0.62824,-0.21882,-0.3784,-0.60484,-0.10613,-0.0065723,0.52873,-0.12537,1.1519,0.16512,-0.12301,0.73452,0.15381,-0.021303,-0.4185,0.32132,-0.56697,0.58886,-0.19564,-0.1671,0.35969,0.42898,-0.071104,0.50165,0.15248,-0.089535,-0.67192,0.1817,-0.030661,-0.20289,-0.23925,0.33899,0.088455,-0.16614,-0.78902,0.14403,-0.22256,-1.377,0.38399,-0.14929,-0.16867,-0.73194,-0.76784,0.96968,0.26804,0.36036,-0.33896,-0.17303,0.3866,-0.76154,-0.52335,0.091027,0.049086,0.059747,0.13165,1.9048,-0.77324,-0.094455,0.17805,-0.096256,0.0684,-0.36729,0.011347,0.12147,0.2453,-0.4354,-0.1733,0.36181,0.78902,0.66458,0.45523,0.079105,0.30238,-0.29991,-0.25161,-0.060112,0.59835,-5.4345e-06,0.47396,-0.25035,-0.16549,0.54022,0.62029,0.43227,0.44656,-0.45033,-0.22726,-0.060223,0.42781,0.34668,-0.38491,-0.25167,1.0969,0.66604,0.036542,-0.32485,0.4289,0.13762,-0.22326,0.69871,0.23841,0.89013,0.23899,-0.51714,-0.050005,0.1487,0.055402,-0.41163,-0.33454,-0.014032,0.36476,-0.23411,-0.13272,-0.050314,0.19593,0.16193,0.28734,0.13478,0.74715,-0.060806,0.14221,0.047109,-0.44866,-0.408,-0.68475,-0.25236,0.16233,-0.33454,0.69021,-0.36958,-0.4338,-0.099908,-0.53847,-0.16861,-0.54684,0.40052,0.11458,0.45688,0.28415,0.43329,0.11012,0.22958,0.024016,0.20695,0.23759,0.0087948,-0.13017,-0.25626]
were = [-0.29716,-0.025477,-0.43389,-0.52616,-0.14354,0.33104,-0.77455,-0.096218,0.15055,0.40654,0.47132,-0.043477,0.018865,-0.31835,0.0021327,0.51416,0.07266,0.86377,0.39094,-0.2941,0.022928,2.9991,-0.14775,-0.48026,-0.46537,-0.28892,-0.28702,-0.78647,0.21987,0.23957,0.2389,0.04237,0.05836,0.11709,-0.18796,-0.31004,-0.64738,-0.7185,0.33694,0.3713,0.6525,-0.48662,0.093115,0.068172,-0.064165,0.0067802,0.8213,-0.46428,0.052333,-0.054842,-0.81224,0.12655,-0.1431,0.53736,-0.10751,-0.1201,-0.13545,0.20551,-0.25494,-0.14893,0.090701,0.36655,-0.20965,0.673,0.51765,-0.35084,-0.47291,0.31541,-0.19238,-0.02999,0.018858,-0.40037,0.0066421,0.12526,0.097123,-0.10354,0.027577,0.073137,0.22532,-0.41189,0.047902,0.11876,-0.57981,0.54385,0.33422,0.3453,0.2162,-0.69258,-0.13744,-0.59278,-0.054606,-0.10773,0.79549,-0.025879,-0.116,0.4545,0.070413,-0.49586,-0.35715,-0.28674,0.5773,-0.81279,0.019403,0.091408,0.13133,0.34139,0.52689,0.64284,-0.26375,-0.018898,-0.51385,-0.062924,0.16246,0.48888,0.72696,0.49494,-0.45197,-0.25742,0.0047182,-0.91796,0.46416,0.1966,0.67564,-0.080587,0.12588,-1.2989,-0.02176,0.4114,0.41177,-0.60598,0.10907,0.0048411,0.36911,-0.35354,0.50438,0.43354,-0.60223,-0.070002,-0.59426,0.081687,-0.74034,-0.071562,-0.14978,-0.47965,1.3329,0.52734,-0.14646,-0.18742,0.23384,-0.088226,-0.22079,0.42821,0.07259,-0.86941,0.065263,0.14209,-0.38875,0.62475,-0.043056,-0.1405,-0.087399,0.26025,0.18315,0.57811,-0.3948,0.84753,0.25191,0.15239,-0.86611,0.56031,-0.13038,0.31795,0.089552,0.24,0.11417,0.13804,-0.47975,-0.49307,-0.65965,-0.03232,0.99141,0.49788,-0.43471,-0.082005,-0.22052,-0.069465,-0.025023,0.40196,-0.22087,0.1288,-0.11603,0.21029,-0.37609,-0.065688,0.72272,0.11403,-0.08957,-0.34873,-0.47252,0.85348]
noti = [0.34303,0.4082,-0.023317,-0.36093,0.0526,0.28925,-0.72928,0.077745,0.25907,0.20004,0.14167,0.49461,-0.043323,-0.17258,0.071147,0.26755,-0.17498,0.81793,0.16388,-0.43131,-0.10978,3.3862,-0.39972,0.079416,0.00044842,0.025372,-0.066779,-0.073348,0.11878,-0.071623,-0.095796,-0.11912,0.13945,0.081686,0.11199,-0.379,-0.86427,-0.65059,0.0072629,0.11515,0.13784,-0.37365,-0.023701,0.31684,-0.22221,0.0094901,0.48885,-0.23512,0.20877,-0.36594,-0.086444,-0.19801,-0.43175,0.22803,0.32309,-0.20011,0.1592,-0.27653,-0.043781,0.30648,0.21574,0.26831,-0.12455,-0.11471,0.29235,-0.041827,-0.27627,0.6043,0.1427,0.5277,0.87699,0.22292,-0.13668,0.13928,-0.46867,-0.13543,-0.47535,-0.46317,0.01747,-0.231,0.14377,-0.15767,0.26873,0.24767,0.24199,-0.12111,-0.40109,-0.5547,0.025637,-1.227,0.25114,0.35073,0.62196,-0.19648,-0.27999,0.09606,-0.082684,-0.014249,-0.099184,-0.11603,0.35344,-0.033989,-0.060309,-0.48616,0.14479,-0.17586,-0.36172,1.2432,-0.4364,-0.16122,-0.13308,-0.34845,0.21167,0.097091,0.23652,0.027138,0.29117,-0.36449,-0.38529,-0.17665,0.37197,0.14412,0.40894,-0.11261,0.097247,-0.24356,-0.018665,0.16607,0.3411,-0.28714,-0.28679,0.49481,0.40374,-0.39017,0.35375,0.12035,-0.065398,-0.16071,-0.044542,-0.28337,0.042688,0.027212,0.37139,-0.30543,1.4214,0.53121,-0.46548,-0.19737,-0.0054895,-0.18203,0.11912,0.42438,-0.40208,0.058662,0.10283,-0.30155,-0.33827,0.34796,-0.082353,0.51862,-0.29775,0.14422,0.44026,-0.123,-0.23571,0.17797,0.11558,0.18801,-0.30945,0.48821,0.1476,0.40829,0.19013,-0.13821,0.333,0.011581,-0.57089,0.30526,0.20618,0.17889,1.2769,-0.11154,-0.47002,-0.12611,-0.20999,-0.013965,-0.078315,0.15576,0.099716,0.27449,-0.36542,0.24097,0.11508,-0.032185,0.18049,-0.028734,0.13449,0.1724,0.031143,0.031783]
this = [0.39086,0.65528,0.064706,-0.33366,0.18502,-0.027321,-0.3878,-0.15081,0.39917,-0.30206,0.23819,0.45941,-0.023606,-0.043237,0.54309,-0.085014,-0.044168,0.66163,-0.39539,-0.27537,0.37465,3.0274,-0.085225,0.1731,0.58574,-0.36105,0.18828,0.41495,0.13081,-0.039031,-0.24917,-0.16286,0.012653,-0.0098054,-0.11815,-0.16429,-0.90413,-0.57109,0.026838,-0.43601,-0.15484,-0.37619,0.24899,0.51744,0.0009744,0.12833,0.24256,0.26005,0.050365,-0.016651,0.091362,-0.31346,-0.0078254,0.72088,0.1415,-0.0050633,-0.24204,-0.39191,-0.058966,-0.053058,0.25604,0.074284,-0.23051,-0.54815,-0.22384,-0.031049,0.019959,0.17193,-0.054222,0.033631,0.39632,0.24702,-0.14935,0.42653,-0.27151,0.28648,-0.46361,-0.21448,-0.46598,0.057568,-0.12724,-0.14651,-0.36591,0.34369,0.05271,-0.018639,-0.46642,-0.49551,0.62577,-0.64458,0.41497,0.15695,0.5569,0.074682,-0.59488,0.15163,0.0050693,-0.11666,0.010334,-0.21172,-0.19726,0.25814,0.16371,0.10557,-0.0063813,0.12384,-0.23964,0.99755,-0.78864,0.25616,0.28623,-0.45224,-0.0022179,0.057452,0.041398,0.09977,0.027042,-0.088172,-0.29811,-0.026336,0.069132,0.23899,0.35541,-0.063479,0.0059764,-0.21982,0.38767,0.1966,-0.027814,-0.088787,-0.19875,0.14134,0.22949,-0.27561,0.13077,0.41609,-0.10151,-0.077741,-0.16479,0.043119,-0.24528,0.21869,0.095889,-0.32395,1.5937,0.31002,-0.058686,-0.45488,0.077925,0.17054,0.0018438,0.71656,-0.42026,0.07127,0.51768,-0.21712,-0.2484,0.22494,0.069718,-0.38438,0.19313,-0.1105,-0.10447,-0.32604,-0.26355,0.073983,-0.27133,0.084472,-0.58727,0.53549,-0.059486,0.22041,0.64939,-0.11922,-0.0081812,-0.063136,-0.15009,-0.096871,-0.27551,0.23581,1.8095,-0.35952,-0.026458,0.47649,-0.18462,0.057494,-0.11701,0.23265,0.043931,0.32839,0.084436,0.051592,0.021732,0.10135,0.075084,-0.23,-0.20108,0.3865,0.052221,-0.22646]
who = [0.075467,-0.29236,-0.26037,-0.28167,0.16097,-0.19472,-0.28206,0.49141,-0.0041119,0.095879,0.33396,0.016874,0.20616,-0.15743,-0.16138,0.21538,-0.14836,0.02546,-0.43681,-0.063661,0.55261,3.0819,-0.11784,-0.46131,-0.27609,0.20948,-0.20544,-0.57155,0.33448,0.15913,0.0025436,0.18004,0.13472,-0.097404,0.35537,-0.47428,-0.79257,-0.54418,0.0243,0.63599,0.12337,-0.12913,-0.26565,-0.24957,-0.52199,-0.40523,0.48403,0.018373,0.23039,0.062138,-0.19292,0.29506,-0.35793,0.16702,0.31868,-0.36054,-0.10978,-0.15632,0.45921,0.096475,-0.377,-0.077615,-0.48899,0.20575,0.50543,0.053419,-0.25978,0.51042,0.097521,0.32606,0.14354,0.0022715,0.48615,0.46938,-0.41124,-0.17148,-0.39744,-0.28901,-0.17756,0.037001,0.3483,0.15934,-0.74281,0.18897,0.043685,0.57208,-0.67016,-0.043947,-0.28336,-0.31996,-0.20404,-0.087898,-0.15724,0.021818,-0.56757,0.63296,-0.10097,-0.065576,0.0058269,0.033035,0.39783,-0.31166,-0.61089,0.27559,0.10008,-0.4199,0.006356,1.8717,0.31473,-0.36004,0.81384,-0.2171,-0.018459,-0.22632,0.14585,-0.1435,-0.041424,0.55974,-0.66752,-0.21959,0.19011,0.33015,0.6129,0.46771,0.42026,-0.52819,0.023165,0.03291,0.47306,0.014006,-0.17396,-0.44362,0.41377,-0.20679,0.39283,0.30211,0.073134,0.042164,-0.9271,-0.47614,0.2431,-0.13379,-0.22238,-0.041457,1.585,0.37481,0.025994,-0.24272,0.30578,0.14687,0.11666,-0.029418,-0.078339,-0.22512,0.13315,-0.064842,-0.28687,-0.01056,-0.34668,0.042145,-0.60041,0.82481,0.31022,0.16489,-0.072921,0.19394,-0.098498,-0.020383,-0.40909,-0.10404,0.19169,-0.15969,0.38026,0.62802,0.2595,-0.33367,-0.73333,-0.40743,0.68423,-0.066338,0.50436,-0.28983,-0.39086,-0.045931,-0.26624,-0.1677,-0.15037,0.14828,-0.28143,-0.17087,-0.25576,-0.056283,-0.1665,0.35106,0.041032,0.27311,0.03002,0.16465,-0.084189,0.057506]
they = [0.0528,0.13495,-0.38214,-0.27999,-0.38392,-0.084598,-0.37277,0.10046,-0.023454,0.71256,0.024259,0.2814,0.010209,-0.19417,-0.30018,0.2074,-0.18105,0.71611,-0.11311,-0.26412,0.17681,3.2366,-0.2329,-0.07999,0.13482,-0.22448,-0.11799,-0.092002,0.23552,-0.12608,0.12557,-0.21536,0.12587,0.049804,0.024991,-0.55934,-0.93924,-0.51797,0.42125,0.34652,0.19897,-0.014204,0.17652,0.43155,-0.38901,0.032722,0.95275,-0.24641,0.12699,0.24171,-0.2495,0.13663,-0.58702,0.53192,0.25364,-0.44631,0.071339,-0.1072,0.016232,0.28356,0.25355,0.36077,-0.066849,0.22595,0.26663,0.1958,-0.21102,0.39934,-0.0066386,0.095334,0.46205,0.028948,0.20202,0.1217,-0.1758,-0.24526,-0.29105,0.153,-0.081641,-0.1527,-0.061132,0.33011,-0.27052,0.24315,0.31077,0.24527,-0.55383,-0.56974,-0.055326,-1.1106,0.23658,0.092638,0.73168,0.085777,-0.24735,0.2301,0.049224,-0.10027,-0.12215,-0.32324,0.45269,-0.2575,-0.16278,-0.18948,0.23016,0.33619,0.11044,1.0879,-0.48341,-0.24421,-0.11329,-0.028835,0.19768,0.31868,0.14758,0.1623,0.071596,0.032118,0.072414,-0.16726,0.74261,0.12729,0.3097,-0.32765,0.12688,-0.69808,0.20057,0.2493,0.49528,-0.4856,-0.46447,0.41582,0.47338,-0.38178,0.4538,0.18398,-0.12377,-0.28023,-0.22348,-0.11058,-0.26895,-0.13358,0.21927,-0.46339,1.4292,0.67536,-0.17342,-0.46714,0.0090229,0.099559,0.15438,0.44203,-0.49867,-0.061944,-0.0033505,-0.17547,-0.1701,0.34289,-0.093336,0.28919,-0.32519,0.13184,0.0011452,-0.084836,-0.26536,0.46792,0.23974,0.044068,-0.75486,0.30871,0.10976,0.039385,0.48202,0.053616,0.43554,0.057253,-0.56428,-0.33534,-0.032656,0.19587,1.2662,-0.3715,-0.74673,-0.19089,-0.2307,0.094298,-0.018306,0.055102,-0.028414,0.27891,-0.22275,0.25638,0.064459,0.031974,0.19601,0.004318,0.043765,0.42737,-0.20698,0.44511]

In [ ]:
print(reviews[0].split())

['the', 'rock', 'is', 'destined', 'to', 'be', 'the', '21st', 'century', 's', 'new', 'conan', 'and', 'that', 'he', 's', 'going', 'to', 'make', 'a', 'splash', 'even', 'greater', 'than', 'arnold', 'schwarzenegger', 'jeanclaud', 'van', 'damme', 'or', 'steven', 'segal']


In [ ]:
a = embeddings_dictionary['movie']
b = embeddings_dictionary['was']
c = embeddings_dictionary['awesome']
d = embeddings_dictionary['it']
e = embeddings_dictionary['was']
f = embeddings_dictionary['horrible']

In [ ]:
print("said = ["+said.replace(" ",",")+']')

said = [-0.13569,0.14029,0.0041988,-0.32062,0.012745,0.92511,-0.44523,-0.16454,0.6016,0.4267,0.26053,0.71426,0.57701,-0.09754,0.64286,-0.0002438,-0.3013,0.097057,-0.21678,-0.27131,0.30927,3.0062,-0.3179,0.28998,-0.39905,0.11234,0.19019,-0.49873,0.11857,-0.22241,-0.52668,-0.040781,-0.16783,-0.35887,0.60394,-0.069027,-0.60611,-0.22444,-0.51665,0.77766,-0.41533,-0.23924,-0.64725,0.16413,-0.41185,-0.15507,0.52085,-0.29298,0.034067,-0.71414,0.17124,0.18186,-0.62824,-0.21882,-0.3784,-0.60484,-0.10613,-0.0065723,0.52873,-0.12537,1.1519,0.16512,-0.12301,0.73452,0.15381,-0.021303,-0.4185,0.32132,-0.56697,0.58886,-0.19564,-0.1671,0.35969,0.42898,-0.071104,0.50165,0.15248,-0.089535,-0.67192,0.1817,-0.030661,-0.20289,-0.23925,0.33899,0.088455,-0.16614,-0.78902,0.14403,-0.22256,-1.377,0.38399,-0.14929,-0.16867,-0.73194,-0.76784,0.96968,0.26804,0.36036,-0.33896,-0.17303,0.3866,-0.76154,-0.52335,0.091027,0.049086,0.059747,0.13165,1.9048,-0.77324,-0.094455,0.17805,-0.096256,0.0684,-0.36729,0.011347,0.

In [ ]:
print(X_train.shape)

(2, 3, 200)


In [ ]:
X_train = np.array([[a,b,c],[d,e,f]])
y_train = np.array([[4,4,4],[0,0,0]])

In [ ]:
import numpy as np
from numpy import savetxt

In [ ]:
#fetch dataset
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
pat='/content/drive/My Drive/Projects/STCS assignment(Word vectors)/'
pat2='/content/drive/My Drive/Projects/HASOC/'


Mounted at /content/drive


In [ ]:
a=np.loadtxt(pat+'Movie Review Dataset.txt',delimiter='\t',dtype=str)

In [ ]:
a2=[]
for e1,e2 in a:
    a2+=[e2]

In [ ]:
a

array(['phrase ids|sentiment values', '0|0.5', '1|0.5', ...,
       '239229|0.33333', '239230|0.88889', '239231|0.5'], dtype='<U27')

In [ ]:
d=np.loadtxt(pat+'Dictionary.txt',delimiter='\t',dtype=str)

In [ ]:
len(d)

239225

In [ ]:
d2=' '.join(d)

In [ ]:
d3=d2.split('|')

In [ ]:
len(d3)

239184

In [ ]:
d3

In [ ]:
d3[-1]

'220444'

In [ ]:
p1=[d3[0]]
p2=[]
for el in d3[1:]:
  e=el.split(' ')
  p1+=[' '.join(e[1:])]
  p2+=[int(e[0])]

In [ ]:
len(p1),len(p2)

(239184, 239183)

In [ ]:
p1[-2]

'É um passatempo descompromissado'

In [ ]:
p1=p1[:-1]

In [ ]:
p2[-1]

220444

In [ ]:
dicti=dict(zip(p2,p1))

In [ ]:
dicti[220444]

'É um passatempo descompromissado'

In [ ]:
p3=[]
for el in a2:
  try:
    p3+=[p2[p1.index(el)]]
  except ValueError:
    p3+=[-1]

In [ ]:
p4=np.array(p3)

In [ ]:
len(p4[p4!=-1])

11285

In [ ]:
sent=np.loadtxt(pat+'Senti_scores.txt',delimiter='|',dtype=str)

In [ ]:
sent[1]

array(['0', '0.5'], dtype='<U16')

In [ ]:
s1=[]
s2=[]
for e1,e2 in sent[1:]:
    s1+=[int(e1)]
    s2+=[float(e2)]

In [ ]:
p6=[]
for el in p3:
  try:
    p6+=[s2[s1.index(el)]]
  except ValueError:
    p6+=[-1]

In [ ]:
p6[3]

0.51389

In [ ]:
#p6 all senti scores in order of sentence if not present has -1
#p3 phrase index of all sentence in that order
